In [1]:
import polars as pl

One of the main problems with messy data is: how do you know if it's messy or not?

We're going to use the NYC 311 service request dataset again here, since it's big and a bit unwieldy. If we try to read it in polars, we immediately run into an error. Polars cannot infer the data type of the data in the csv file:

In [2]:
requests = pl.read_csv('../data/311-service-requests.csv')

ComputeError: Could not parse `11549-3650` as dtype `i64` at column 'Incident Zip' (column number 9).
The current offset in the file is 34985879 bytes.

You might want to try:
- increasing `infer_schema_length` (e.g. `infer_schema_length=10000`),
- specifying correct dtype with the `dtypes` argument
- setting `ignore_errors` to `True`,
- adding `11549-3650` to the `null_values` list.

We can force polars to try harder to infer the data type by setting `infer_schema_length` to `None`. Looking at the schema below, I can see that `Incident Zip` was parsed as a `Utf8` (a string). That doesn't look right.

In [3]:
requests = pl.read_csv('../data/311-service-requests.csv', infer_schema_length=None)
display(requests.head())
display(requests.schema)

Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type,Status,Due Date,Resolution Action Updated Date,Community Board,Borough,X Coordinate (State Plane),Y Coordinate (State Plane),Park Facility Name,Park Borough,School Name,School Number,School Region,School Code,School Phone Number,School Address,School City,School State,School Zip,School Not Found,School or Citywide Complaint,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location
i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,str
26589651,"""10/31/2013 02:…",null,"""NYPD""","""New York City …","""Noise - Street…","""Loud Talking""","""Street/Sidewal…","""11432""","""90-03 169 STRE…","""169 STREET""","""90 AVENUE""","""91 AVENUE""",null,null,"""ADDRESS""","""JAMAICA""",null,"""Precinct""","""Assigned""","""10/31/2013 10:…","""10/31/2013 02:…","""12 QUEENS""","""QUEENS""",1042027,197389,"""Unspecified""","""QUEENS""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""N""",null,null,null,null,null,null,null,null,null,null,null,40.708275,-73.791604,"""(40.7082753259…"
26593698,"""10/31/2013 02:…",null,"""NYPD""","""New York City …","""Illegal Parkin…","""Commercial Ove…","""Street/Sidewal…","""11378""","""58 AVENUE""","""58 AVENUE""","""58 PLACE""","""59 STREET""",null,null,"""BLOCKFACE""","""MASPETH""",null,"""Precinct""","""Open""","""10/31/2013 10:…",null,"""05 QUEENS""","""QUEENS""",1009349,201984,"""Unspecified""","""QUEENS""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""N""",null,null,null,null,null,null,null,null,null,null,null,40.721041,-73.909453,"""(40.7210405356…"
26594139,"""10/31/2013 02:…","""10/31/2013 02:…","""NYPD""","""New York City …","""Noise - Commer…","""Loud Music/Par…","""Club/Bar/Resta…","""10032""","""4060 BROADWAY""","""BROADWAY""","""WEST 171 STREE…","""WEST 172 STREE…",null,null,"""ADDRESS""","""NEW YORK""",null,"""Precinct""","""Closed""","""10/31/2013 10:…","""10/31/2013 02:…","""12 MANHATTAN""","""MANHATTAN""",1001088,246531,"""Unspecified""","""MANHATTAN""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""N""",null,null,null,null,null,null,null,null,null,null,null,40.84333,-73.939144,"""(40.8433297546…"
26595721,"""10/31/2013 01:…","""10/31/2013 02:…","""NYPD""","""New York City …","""Noise - Vehicl…","""Car/Truck Horn…","""Street/Sidewal…","""10023""","""WEST 72 STREET…","""WEST 72 STREET…","""COLUMBUS AVENU…","""AMSTERDAM AVEN…",null,null,"""BLOCKFACE""","""NEW YORK""",null,"""Precinct""","""Closed""","""10/31/2013 09:…","""10/31/2013 02:…","""07 MANHATTAN""","""MANHATTAN""",989730,222727,"""Unspecified""","""MANHATTAN""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""N""",null,null,null,null,null,null,null,null,null,null,null,40.778009,-73.980213,"""(40.7780087446…"
26590930,"""10/31/2013 01:…",null,"""DOHMH""","""Department of …","""Rodent""","""Condition Attr…","""Vacant Lot""","""10027""","""WEST 124 STREE…","""WEST 124 STREE…","""LENOX AVENUE""","""ADAM CLAYTON P…",null,null,"""BLOCKFACE""","""NEW YORK""",null,"""N/A""","""Pending""","""11/30/2013 01:…","""10/31/2013 01:…","""1

{'Unique Key': Int64,
 'Created Date': Utf8,
 'Closed Date': Utf8,
 'Agency': Utf8,
 'Agency Name': Utf8,
 'Complaint Type': Utf8,
 'Descriptor': Utf8,
 'Location Type': Utf8,
 'Incident Zip': Utf8,
 'Incident Address': Utf8,
 'Street Name': Utf8,
 'Cross Street 1': Utf8,
 'Cross Street 2': Utf8,
 'Intersection Street 1': Utf8,
 'Intersection Street 2': Utf8,
 'Address Type': Utf8,
 'City': Utf8,
 'Landmark': Utf8,
 'Facility Type': Utf8,
 'Status': Utf8,
 'Due Date': Utf8,
 'Resolution Action Updated Date': Utf8,
 'Community Board': Utf8,
 'Borough': Utf8,
 'X Coordinate (State Plane)': Int64,
 'Y Coordinate (State Plane)': Int64,
 'Park Facility Name': Utf8,
 'Park Borough': Utf8,
 'School Name': Utf8,
 'School Number': Utf8,
 'School Region': Utf8,
 'School Code': Utf8,
 'School Phone Number': Utf8,
 'School Address': Utf8,
 'School City': Utf8,
 'School State': Utf8,
 'School Zip': Utf8,
 'School Not Found': Utf8,
 'School or Citywide Complaint': Utf8,
 'Vehicle Type': Utf8,
 'Taxi

# 7.1 How do we know if it's messy? 

We're going to look at a few columns here. I know already that there are some problems with the zip code, so let's look at that first.
 
To get a sense for whether a column has problems, I usually use `.unique()` to look at all its values. If it's a numeric column, I'll instead plot a histogram to get a sense of the distribution.

When we look at the unique values in "Incident Zip", it quickly becomes clear that this is a mess.

Some of the problems:

* Some have been parsed as strings, and some as floats
* There are `nan`s 
* Some of the zip codes are `29616-0759` or `83`
* There are some N/A values that polars didn't recognize, like 'N/A' and 'NO CLUE'

What we can do:

* Normalize 'N/A' and 'NO CLUE' into regular nan values
* Look at what's up with the 83, and decide what to do
* Make everything strings

In [4]:
requests['Incident Zip'].unique().sort()

Incident Zip
str
null
"""00000"""
"""000000"""
"""00083"""
"""02061"""
"""06901"""
"""07020"""
"""07087"""
"""07093"""


# 7.2 Fixing the null_values and string/float confusion

We can pass a `null_values` option to `pl.read_csv` to clean this up a little bit. We can also specify that the type of Incident Zip is a string, not a float.

In [5]:
null_values = ['NO CLUE', 'N/A', '0', 'NA']
requests = pl.read_csv('../data/311-service-requests.csv', null_values=null_values, dtypes={'Incident Zip':pl.Utf8})
requests['Incident Zip'].unique().sort()

Incident Zip
str
null
"""00000"""
"""000000"""
"""00083"""
"""02061"""
"""06901"""
"""07020"""
"""07087"""
"""07093"""


# 7.3 What's up with the dashes?

In [6]:
rows_with_dashes = requests.filter(
    pl.col('Incident Zip').str.contains('-')
)
print('number of zip codes with dashes: ', rows_with_dashes.height)
rows_with_dashes.head()

number of zip codes with dashes:  5


Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type,Status,Due Date,Resolution Action Updated Date,Community Board,Borough,X Coordinate (State Plane),Y Coordinate (State Plane),Park Facility Name,Park Borough,School Name,School Number,School Region,School Code,School Phone Number,School Address,School City,School State,School Zip,School Not Found,School or Citywide Complaint,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location
i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,str
26550551,"""10/24/2013 06:…",null,"""DCA""","""Department of …","""Consumer Compl…","""False Advertis…",null,"""77092-2016""","""2700 EAST SELT…","""EAST SELTICE W…",null,null,null,null,null,"""HOUSTON""",null,null,"""Assigned""","""11/13/2013 11:…","""10/29/2013 11:…","""0 Unspecified""","""Unspecified""",null,null,"""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""N""",null,null,null,null,null,null,null,null,null,null,null,null,null,null
26548831,"""10/24/2013 09:…",null,"""DCA""","""Department of …","""Consumer Compl…","""Harassment""",null,"""55164-0737""","""P.O. BOX 64437…","""64437""",null,null,null,null,null,"""ST. PAUL""",null,null,"""Assigned""","""11/13/2013 02:…","""10/29/2013 02:…","""0 Unspecified""","""Unspecified""",null,null,"""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""N""",null,null,null,null,null,null,null,null,null,null,null,null,null,null
26488417,"""10/15/2013 03:…",null,"""TLC""","""Taxi and Limou…","""Taxi Complaint…","""Driver Complai…","""Street""","""11549-3650""","""365 HOFSTRA UN…","""HOFSTRA UNIVER…",null,null,null,null,null,"""HEMSTEAD""",null,null,"""Assigned""","""11/30/2013 01:…","""10/16/2013 01:…","""0 Unspecified""","""Unspecified""",null,null,"""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""N""",null,null,null,"""La Guardia Air…",null,null,null,null,null,null,null,null,null,null
26468296,"""10/10/2013 12:…","""10/26/2013 01:…","""DCA""","""Department of …","""Consumer Compl…","""Debt Not Owed""",null,"""29616-0759""","""PO BOX 25759""","""BOX 25759""",null,null,null,null,null,"""GREENVILLE""",null,null,"""Closed""","""10/26/2013 09:…","""10/26/2013 01:…","""0 Unspecified""","""Unspecified""",null,null,"""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""N""",null,null,null,null,null,null,null,null,null,null,null,null,null,null
26461137,"""10/09/2013 05:…","""10/25/2013 01:…","""DCA""","""Department of …","""Consumer Compl…","""Harassment""",null,"""35209-3114""","""600 BEACON PKW…","""BEACON PKWY""",null,null,null,null,null,"""BIRMINGHAM""",null,null,"""Closed""","""10/25/2013 02:…","""10/25/2013 01:…","""0 Unspecified""","""Unspecified""",null,null,"""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""N""",null,null,null,null,null,null,null,null,null,null,null,null,null

I thought these were missing data and originally deleted them. But then my friend Dave pointed out that 9-digit zip codes are normal. Let's look at all the zip codes with more than 5 digits, make sure they're okay, and then truncate them.

In [7]:
requests.filter(
    pl.col('Incident Zip').str.contains('-')
)['Incident Zip'].unique()

Incident Zip
str
"""77092-2016"""
"""29616-0759"""
"""55164-0737"""
"""35209-3114"""
"""11549-3650"""


Those all look okay to truncate to me.

In [8]:
requests = requests.with_columns(
    pl.col('Incident Zip').str.slice(0, 5)
)
requests.filter(
    pl.col('Incident Zip').str.contains('-')
)['Incident Zip'].unique()

Incident Zip
str


Done.

Earlier I thought 00083 was a broken zip code, but turns out Central Park's zip code 00083! Shows what I know. I'm still concerned about the 00000 zip codes, though: let's look at that. 

In [9]:
requests.filter(
    pl.col('Incident Zip') == '00000'
)

Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type,Status,Due Date,Resolution Action Updated Date,Community Board,Borough,X Coordinate (State Plane),Y Coordinate (State Plane),Park Facility Name,Park Borough,School Name,School Number,School Region,School Code,School Phone Number,School Address,School City,School State,School Zip,School Not Found,School or Citywide Complaint,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location
i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,str
26529313,"""10/22/2013 02:…",null,"""TLC""","""Taxi and Limou…","""Taxi Complaint…","""Driver Complai…",null,"""00000""","""EWR EWR""","""EWR""",null,null,null,null,null,"""NEWARK""",null,null,"""Assigned""","""12/07/2013 09:…","""10/23/2013 09:…","""0 Unspecified""","""Unspecified""",null,null,"""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""N""",null,null,null,"""Other""",null,null,null,null,null,null,null,null,null,null
26507389,"""10/17/2013 05:…",null,"""TLC""","""Taxi and Limou…","""Taxi Complaint…","""Driver Complai…","""Street""","""00000""","""1 NEWARK AIRPO…","""NEWARK AIRPORT…",null,null,null,null,null,"""NEWARK""",null,null,"""Assigned""","""12/02/2013 11:…","""10/18/2013 12:…","""0 Unspecified""","""Unspecified""",null,null,"""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""Unspecified""","""N""",null,null,null,"""Other""",null,null,null,null,null,null,null,null,null,null


This looks bad to me. Let's set these to nan.

In [10]:
requests = requests.with_columns(
    pl.col('Incident Zip').map_dict({'00000': None}, default=pl.first())
)
requests.filter(
    pl.col('Incident Zip') == '00000'
).height

0

Great. Let's see where we are now:

In [11]:
unique_zips = requests['Incident Zip'].unique().sort()
unique_zips

Incident Zip
str
null
"""00083"""
"""02061"""
"""06901"""
"""07020"""
"""07087"""
"""07093"""
"""07109"""
"""07114"""


Amazing! This is much cleaner. There's something a bit weird here, though -- I looked up 77056 on Google maps, and that's in Texas.

Let's take a closer look:

In [12]:
is_close = pl.col('Incident Zip').str.starts_with('0') | pl.col('Incident Zip').str.starts_with('1')
requests.lazy().select(
    'Incident Zip',
    'Descriptor',
    'City'
).filter(
    ~is_close & pl.col('Incident Zip').is_not_null()
).sort('Incident Zip').collect()

Incident Zip,Descriptor,City
str,str,str
"""23502""","""Harassment""","""NORFOLK"""
"""23541""","""Harassment""","""NORFOLK"""
"""29616""","""Debt Not Owed""","""GREENVILLE"""
"""35209""","""Harassment""","""BIRMINGHAM"""
"""41042""","""Harassment""","""FLORENCE"""
"""55164""","""Harassment""","""ST. PAUL"""
"""61702""","""Billing Disput…","""BLOOMIGTON"""
"""70711""","""Contract Dispu…","""CLIFTON"""
"""77056""","""Debt Not Owed""","""HOUSTON"""


Okay, there really are requests coming from LA and Houston! Good to know. Filtering by zip code is probably a bad way to handle this -- we should really be looking at the city instead.

In [13]:
requests['City'].str.to_uppercase().value_counts(sort=True)

City,counts
str,u32
"""BROOKLYN""",31662
"""NEW YORK""",22664
"""BRONX""",18438
null,12215
"""STATEN ISLAND""",4766
"""JAMAICA""",2246
"""FLUSHING""",1803
"""ASTORIA""",1568
"""RIDGEWOOD""",1073


There are 12,215 `null` values in the `City` column. Upon closer look, it seems that many of these rows also have missing `Incident Zip` values as well:

In [14]:
requests.select(
    'Incident Zip',
    'Descriptor',
    'City'
).filter(
    pl.col('City').is_null()
).sort('Incident Zip')

Incident Zip,Descriptor,City
str,str,str
null,"""Street Light O…",null
null,"""Street Light O…",null
null,"""Medicaid""",null
null,"""Controller""",null
null,"""Property Tax E…",null
null,"""Ped Flasher""",null
null,"""Cash Assistanc…",null
null,"""Property Tax E…",null
null,"""Property Tax E…",null


# 7.4 Putting it together

Here's what we ended up doing to clean up our zip codes, all together:

In [15]:
null_values = ['NO CLUE', 'N/A', '0', 'NA']
requests = pl.scan_csv('../data/311-service-requests.csv', null_values=null_values, dtypes={'Incident Zip':pl.Utf8})
requests = (requests
            .with_columns(
                pl.col('Incident Zip').str.slice(0, 5).map_dict({'00000': None}, default=pl.first())
            ).filter(
                pl.col('Incident Zip').is_not_null()
            ).collect())

In [16]:
requests['Incident Zip'].unique().sort()

Incident Zip
str
"""00083"""
"""02061"""
"""06901"""
"""07020"""
"""07087"""
"""07093"""
"""07109"""
"""07114"""
"""07201"""


<style>
    @font-face {
        font-family: "Computer Modern";
        src: url('http://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');
    }
    div.cell{
        width:800px;
        margin-left:16% !important;
        margin-right:auto;
    }
    h1 {
        font-family: Helvetica, serif;
    }
    h4{
        margin-top:12px;
        margin-bottom: 3px;
       }
    div.text_cell_render{
        font-family: Computer Modern, "Helvetica Neue", Arial, Helvetica, Geneva, sans-serif;
        line-height: 145%;
        font-size: 130%;
        width:800px;
        margin-left:auto;
        margin-right:auto;
    }
    .CodeMirror{
            font-family: "Source Code Pro", source-code-pro,Consolas, monospace;
    }
    .text_cell_render h5 {
        font-weight: 300;
        font-size: 22pt;
        color: #4057A1;
        font-style: italic;
        margin-bottom: .5em;
        margin-top: 0.5em;
        display: block;
    }
    
    .warning{
        color: rgb( 240, 20, 20 )
        }  